In [ ]:
!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'

Importar Librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import requests

In [ ]:
global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision

In [ ]:
ticker = "BTC-USD"
start_date = pd.Timestamp.now() - pd.Timedelta(days=7)
end_date = pd.Timestamp.now()

df_bitcoin = yf.download(ticker, start=start_date, end=end_date, interval="5m")
df_bitcoin.info()

In [ ]:
df_bitcoin_limpio = df_bitcoin.copy()
df_bitcoin_limpio

#Eliminar los Duplicados de la columna Datetime

In [ ]:
df_bitcoin_limpio = df_bitcoin_limpio.drop_duplicates()
df_bitcoin_limpio

#Contar los valores nulos en la columna Close y Volume


In [91]:
valores_nulos_C = df_bitcoin_limpio.Close.isnull().sum()
valores_nulos_C

0

In [92]:
valores_nulos_V = df_bitcoin_limpio.Volume.isnull().sum()
valores_nulos_V

0

#Contar y eliminar los valores en cero en la columna Volume

In [99]:
valores_cero_V = (df_bitcoin_limpio.Volume == 0).sum()
valores_cero_V

1002

In [ ]:
df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio.Volume != 0]
df_bitcoin_limpio

In [ ]:
df_bitcoin_limpio.describe().round(2)

In [ ]:
pd.DataFrame(df_bitcoin_limpio.Close).boxplot();

#Filtra o selecciona únicamente los registros cuyo precio(Close) se encuentren entre el 1er cuartil(Q1) y el 3er cuartil(Q3) del boxplot.

In [ ]:
q1 = df_bitcoin_limpio.Close.quantile(0.25)
q3 = df_bitcoin_limpio.Close.quantile(0.75)

# Filtramos el DataFrame utilizando los cuartiles
df_bitcoin_limpio = df_bitcoin_limpio[(df_bitcoin_limpio.Close >= q1) & (df_bitcoin_limpio.Close <= q3)]

df_bitcoin_limpio

#Finalmente, calcula el precio promedio(Close) de esta selección y guárdala en la variable media_bitcoin.

In [119]:
pd.DataFrame(df_bitcoin_limpio.Close).mean()

Close    28422.756521
dtype: float64

In [ ]:
fig, ax = plt.subplots(figsize = (20,5))
ax = df_bitcoin.Open.plot(fontsize = 14)
ax.axhline(y=np.mean(df_bitcoin.Open), ls = "--")
ax.set_title("Valor Bitcoin", fontsize = 20)
ax.set_ylabel("Bitcoin USD", fontsize = 16)
ax.set_xlabel("Dias", fontsize = 16);

In [ ]:
df_bitcoin_limpio_C = pd.DataFrame(df_bitcoin_limpio.Close)
df_bitcoin_limpio_C.boxplot();

In [ ]:
def extraer_tendencias():
    global precio_actual, tendencia
    
    # URL de la página de CoinMarketCap
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
    url = "https://coinmarketcap.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Buscar el precio actual del Bitcoin en la página
    precio_element = soup.find("a", {"href": "/currencies/bitcoin/"})
    precio_actual = float(precio_element.find("div", {"class": "sc-cadad039-0 clgqXO"}).text.replace(",", ""))

    # Buscar la variación del precio en la última hora
    variacion_element = soup.find("a", {"href": "/currencies/bitcoin/"})
    variacion_1h = float(variacion_element.find("span", {"class": "sc-15yy2pl-0 hVAiNH priceChangePercent___1hjJK"}).text.replace("%", ""))

    # Determinar la tendencia según la variación del precio en la última hora
    if variacion_1h < 0:
        tendencia = "baja"
    else:
        tendencia = "alta"
        
    print("El precio actual del Bitcoin es de", precio_actual, "USD")
    print("La tendencia es", tendencia)

In [82]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
url = "https://coinmarketcap.com/"
response = requests.get(url, headers = headers)
soup = BeautifulSoup(response.content, "html.parser")

precio_actual = soup.find("div", {"class": "sc-cadad039-0 clgqXO"}).text.replace(",","").replace("$","")
variacion_1h = soup.find("span", {"class": "sc-97d6d2ca-0 bQjSqS"}).text.replace("%","")
#variacion_1h_2 = soup.find("span", {"class": "icon-Caret-down"}).text
precio_actual = float(precio_actual)
variacion_1h = float(variacion_1h)
print(precio_actual)
print(variacion_1h)
print(variacion_1h_2)

27591.13
0.52



In [45]:
type(variacion_1h)

float